In [ ]:
!pip install evaluate
!pip install -U transformers
!pip install accelerate -U

In [ ]:
from datasets import Dataset
import requests
import pandas as pd
import io
from transformers import AutoModelForSequenceClassification
import re
import random
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
from transformers import pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import os
os.environ["WANDB_DISABLED"] = "true"





tinyTest=False


url = 'https://raw.githubusercontent.com/bitext/customer-support-llm-chatbot-training-dataset/main/data/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv'

response = requests.get(url)

if response.status_code == 200:

    csv_data = io.BytesIO(response.content)
    df = pd.read_csv(csv_data)
    print(df.head())  # Display the first few rows of the DataFrame
else:
    print("Failed to download the CSV file.")

print("Downloaded ",len(df),"rows")

df=df.drop('flags',axis=1)
origdf =df.copy()




In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
city_names = ["Aberdeen", "Abilene", "Akron", "Albany", "Albuquerque", "Alexandria", "Allentown", "Amarillo", "Anaheim", "Anchorage", "Ann Arbor", "Antioch", "Apple Valley", "Appleton", "Arlington", "Arvada", "Asheville", "Athens", "Atlanta", "Atlantic City", "Augusta", "Aurora", "Austin", "Bakersfield", "Baltimore", "Barnstable", "Baton Rouge", "Beaumont", "Bel Air", "Bellevue", "Berkeley", "Bethlehem", "Billings", "Birmingham", "Bloomington", "Boise", "Boise City", "Bonita Springs", "Boston", "Boulder", "Bradenton", "Bremerton", "Bridgeport", "Brighton", "Brownsville", "Bryan", "Buffalo", "Burbank", "Burlington", "Cambridge", "Canton", "Cape Coral", "Carrollton", "Cary", "Cathedral City", "Cedar Rapids", "Champaign", "Chandler", "Charleston", "Charlotte", "Chattanooga", "Chesapeake", "Chicago", "Chula Vista", "Cincinnati", "Clarke County", "Clarksville", "Clearwater", "Cleveland", "College Station", "Colorado Springs", "Columbia", "Columbus", "Concord", "Coral Springs", "Corona", "Corpus Christi", "Costa Mesa", "Dallas", "Daly City", "Danbury", "Davenport", "Davidson County", "Dayton", "Daytona Beach", "Deltona", "Denton", "Denver", "Des Moines", "Detroit", "Downey", "Duluth", "Durham", "El Monte", "El Paso", "Elizabeth", "Elk Grove", "Elkhart", "Erie", "Escondido", "Eugene", "Evansville", "Fairfield", "Fargo", "Fayetteville", "Fitchburg", "Flint", "Fontana", "Fort Collins", "Fort Lauderdale", "Fort Smith", "Fort Walton Beach", "Fort Wayne", "Fort Worth", "Frederick", "Fremont", "Fresno", "Fullerton", "Gainesville", "Garden Grove", "Garland", "Gastonia", "Gilbert", "Glendale", "Grand Prairie", "Grand Rapids", "Grayslake", "Green Bay", "GreenBay", "Greensboro", "Greenville", "Gulfport-Biloxi", "Hagerstown", "Hampton", "Harlingen", "Harrisburg", "Hartford", "Havre de Grace", "Hayward", "Hemet", "Henderson", "Hesperia", "Hialeah", "Hickory", "High Point", "Hollywood", "Honolulu", "Houma", "Houston", "Howell", "Huntington", "Huntington Beach", "Huntsville", "Independence", "Indianapolis", "Inglewood", "Irvine", "Irving", "Jackson", "Jacksonville", "Jefferson", "Jersey City", "Johnson City", "Joliet", "Kailua", "Kalamazoo", "Kaneohe", "Kansas City", "Kennewick", "Kenosha", "Killeen", "Kissimmee", "Knoxville", "Lacey", "Lafayette", "Lake Charles", "Lakeland", "Lakewood", "Lancaster", "Lansing", "Laredo", "Las Cruces", "Las Vegas", "Layton", "Leominster", "Lewisville", "Lexington", "Lincoln", "Little Rock", "Long Beach", "Lorain", "Los Angeles", "Louisville", "Lowell", "Lubbock", "Macon", "Madison", "Manchester", "Marina", "Marysville", "McAllen", "McHenry", "Medford", "Melbourne", "Memphis", "Merced", "Mesa", "Mesquite", "Miami", "Milwaukee", "Minneapolis", "Miramar", "Mission Viejo", "Mobile", "Modesto", "Monroe", "Monterey", "Montgomery", "Moreno Valley", "Murfreesboro", "Murrieta", "Muskegon", "Myrtle Beach", "Naperville", "Naples", "Nashua", "Nashville", "New Bedford", "New Haven", "New London", "New Orleans", "New York", "New York City", "Newark", "Newburgh", "Newport News", "Norfolk", "Normal", "Norman", "North Charleston", "North Las Vegas", "North Port", "Norwalk", "Norwich", "Oakland", "Ocala", "Oceanside", "Odessa", "Ogden", "Oklahoma City", "Olathe", "Olympia", "Omaha", "Ontario", "Orange", "Orem", "Orlando", "Overland Park", "Oxnard", "Palm Bay", "Palm Springs", "Palmdale", "Panama City", "Pasadena", "Paterson", "Pembroke Pines", "Pensacola", "Peoria", "Philadelphia", "Phoenix", "Pittsburgh", "Plano", "Pomona", "Pompano Beach", "Port Arthur", "Port Orange", "Port Saint Lucie", "Port St. Lucie", "Portland", "Portsmouth", "Poughkeepsie", "Providence", "Provo", "Pueblo", "Punta Gorda", "Racine", "Raleigh", "Rancho Cucamonga", "Reading", "Redding", "Reno", "Richland", "Richmond", "Richmond County", "Riverside", "Roanoke", "Rochester", "Rockford", "Roseville", "Round Lake Beach", "Sacramento", "Saginaw", "Saint Louis", "Saint Paul", "Saint Petersburg", "Salem", "Salinas", "Salt Lake City", "San Antonio", "San Bernardino", "San Buenaventura", "San Diego", "San Francisco", "San Jose", "Santa Ana", "Santa Barbara", "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Maria", "Santa Rosa", "Sarasota", "Savannah", "Scottsdale", "Scranton", "Seaside", "Seattle", "Sebastian", "Shreveport", "Simi Valley", "Sioux City", "Sioux Falls", "South Bend", "South Lyon", "Spartanburg", "Spokane", "Springdale", "Springfield", "St. Louis", "St. Paul", "St. Petersburg", "Stamford", "Sterling Heights", "Stockton", "Sunnyvale", "Syracuse", "Tacoma", "Tallahassee", "Tampa", "Temecula", "Tempe", "Thornton", "Thousand Oaks", "Toledo", "Topeka", "Torrance", "Trenton", "Tucson", "Tulsa", "Tuscaloosa", "Tyler", "Utica", "Vallejo", "Vancouver", "Vero Beach", "Victorville", "Virginia Beach", "Visalia", "Waco", "Warren", "Washington", "Waterbury", "Waterloo", "West Covina", "West Valley City", "Westminster", "Wichita", "Wilmington", "Winston", "Winter Haven", "Worcester", "Yakima", "Yonkers", "York", "Youngstown"]
country_names = [
    "United States",    # North America
    "Canada",           # North America
    "Mexico",           # North America
    "United Kingdom",   # Europe
    "Germany",          # Europe
    "France",           # Europe
    "Spain",            # Europe
    "Italy",            # Europe
    "Netherlands",      # Europe
    "Sweden"            # Europe
]

currency_symbols = [
    "$", 
    "£"
]

last_names = [
    "Smith",
    "Johnson",
    "Williams",
    "Brown",
    "Jones",
    "Garcia",
    "Miller",
    "Davis",
    "Rodriguez",
    "Martinez"
]

first_names = [
    "James",
    "John",
    "Robert",
    "Michael",
    "William",
    "David",
    "Richard",
    "Joseph",
    "Charles",
    "Thomas",
    "Daniel",
    "Matthew",
    "Jennifer",
    "Jessica",
    "Ashley",
    "Sarah",
    "Emily",
    "Samantha",
    "Elizabeth",
    "Emma"
]


accounts =['Checking','Savings','Spending','Prepaid']



In [ ]:
targetsToFill = df.instruction.str.extract(r'\{\{([^}]*)\}\}')
targetsToFill.value_counts()


In [ ]:
#df['instruction']=df.instruction

In [ ]:

df[['category','intent']].value_counts()

labelNames = pd.DataFrame(df['intent'].value_counts()).reset_index().drop("intent",axis=1)
labelNames.columns=['labelNames']
labelDict = labelNames.to_dict()['labelNames']
labelDict = {v: k for k, v in labelDict.items()}
reverseLabel = {c:k for k,c in labelDict.items()}

In [ ]:
modelName = 'distilbert-base-uncased'

In [ ]:
df['label']=df.intent.map(labelDict)

df.head()


In [ ]:
df['textlen']=df['instruction'].str.len()
print(df.textlen.describe())
df.drop("textlen",axis=1,inplace=True)

In [ ]:


def slugOrderNumber(s,t ='{{Order Number}}' ):
    oNum = str(random.randint(100,1000000))+'-'+str(random.randint(100,1000000))
    ss=re.sub(t,oNum,s)
    return ss

def slugInvoiceNumber(s,t ='{{Invoice Number}}' ):
    oNum = str(random.randint(100,1000000))+'-'+str(random.randint(100,1000000))
    ss=re.sub(t,oNum,s)
    return ss

def slugRefundAmount(s,t ='{{Refund Amount}}' ):
    oNum = str(round(random.uniform(5.,250),2))
    ss=re.sub(t,oNum,s)
    return ss

def slugMoneyAmount(s,t ='{{Money Amount}}' ):
    oNum = str(round(random.uniform(5.,250),2))
    ss=re.sub(t,oNum,s)
    return ss

def slugDeliveryCity(s,t ='{{Delivery City}}' ):
    newCity = random.choice(city_names)
    ss=re.sub(t,newCity,s)
    return ss

def slugDeliveryCountry(s,t ='{{Delivery Country}}' ):
    newCountry = random.choice(country_names)
    ss=re.sub(t,newCountry,s)
    return ss

def slugDeliveryAccountType(s,t ='{{Account Type}}' ):
    newAccount = random.choice(accounts)
    ss=re.sub(t,newAccount,s)
    return ss

def slugDeliveryAccountCategory(s,t ='{{Account Category}}' ):
    newAccount = random.choice(accounts)
    ss=re.sub(t,newAccount,s)
    return ss

def slugCurrencySymbol(s,t ='{{Currency Symbol}}' ):
    newCurrency = random.choice(currency_symbols)
    ss=re.sub(t,newCurrency,s)
    return ss

def slugPersonName(s,t ='{{Person Name}}' ):
    fName = random.choice(first_names)
    lName = random.choice(last_names)
    ss=re.sub(t,fName+' '+lName,s)
    return ss

dataFunctions = ['slugOrderNumber',
 'slugInvoiceNumber',
 'slugRefundAmount',
 'slugMoneyAmount',
 'slugDeliveryCity',
 'slugDeliveryCountry',
 'slugDeliveryAccountType',
 'slugDeliveryAccountCategory',
 'slugCurrencySymbol',
 'slugPersonName']

for f in dataFunctions:
    df['instruction']=df['instruction'].apply(locals()[f])

targetsToFill = df.instruction.str.extract(r'\{\{([^}]*)\}\}')
unique_targets = targetsToFill.value_counts()
if len(unique_targets)==0:
    print("Great! All targets are filled with data")
else:
    print("Still need to fix these!")
    print(unique_targets.head(20))




In [ ]:

df = df.sample(n=1300,replace=False,random_state=42)

df_train, df_test  = train_test_split(df, test_size=0.33, random_state=42)
if tinyTest:
    df_train, junk  = train_test_split(df_train, test_size=0.99, random_state=42)
df_test, df_val  = train_test_split(df_test, test_size=0.5, random_state=42)
if tinyTest:
    df_test, junk  = train_test_split(df_test, test_size=0.99, random_state=42)
    df_val, junk  = train_test_split(df_val, test_size=0.99, random_state=42)

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)
val_dataset = Dataset.from_pandas(df_val)

print(len(train_dataset),len(test_dataset),len(val_dataset))




In [ ]:


tokenizer = AutoTokenizer.from_pretrained(modelName)


def tokenize_function(examples):
    return tokenizer(examples["instruction"], padding="max_length", truncation=True)


tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)
#tokenized_val= val_dataset.map(tokenize_function, batched=True)

In [ ]:


model = AutoModelForSequenceClassification.from_pretrained(modelName, num_labels=len(labelDict))

In [ ]:


training_args = TrainingArguments(output_dir="test_trainer")

if tinyTest:
    training_args.num_epochs=1




metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:

training_args.num_train_epochs=10


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)


In [ ]:
#Score the full test, then run a labeled confusion matrix
#Use existing model to gauge sentiment - do a dist vs. actual and pred labels



In [ ]:


trainer.train()

In [ ]:
if "custServiceClassifier" not in os.listdir():
    os.mkdir("custServiceClassifier")
    
trainer.save_model("custServiceClassifier/")

In [ ]:



clf = pipeline("text-classification", model,tokenizer =modelName )

In [ ]:
z=clf.predict(val_dataset['instruction'])

In [ ]:
y=pd.DataFrame(z)
y.columns = ['predTxt','score']
y['true']=val_dataset['label']
#y['pred']=y.predTxt.str[6:].astype(int)
y


In [ ]:

accuracy_score(y.pred,y.true)

In [ ]:
pd.set_option('display.max_columns', 25)
cm=pd.DataFrame(confusion_matrix(y.pred,y.true,normalize='true')).round(1)


import seaborn as sns
%matplotlib inline


#df = DataFrame(abs(np.random.randn(5, 4)), index=Index, columns=Cols)

sns.heatmap(cm, annot=False)

In [ ]:
model.config.id2label=reverseLabel
model.push_to_hub("custServiceClassifier")

clf = pipeline("text-classification", model,tokenizer =modelName )

clf.predict("Hey, I'm really pissed. What's the status of getting my money back?  You guys screwed it up royally.")